In [22]:
import numpy as np
import secrets # get random value from system
# secrets.randbelow(100)

Unicode convert

In [26]:
# converts a string to a numpy array of unicode indexes
def stringToUnicode(input):
    result =[]
    chars = list(input)
    for c in chars:
        result.append(ord(c))
    return np.array(result)

# converts a numpy array of unicode indexes to a string
def unicodeToString(input):
    result =''
    for u in input:
        result = result +str((chr(u)))
    return result

def generate_key(plain, n=256):
    key = []
    for _ in range(len(plain)):
        key.append(secrets.randbelow(n))
    key = unicodeToString(np.array(key))
    return key

#### Cesar cipher encryption & decryption

$k(x) = (x + k) \mod n$

$k^{-1}(x) = (x - k) \mod n$

In [12]:
def cesar_cipher_encryption(text, key:int, n=1114112):
  unicodes = (stringToUnicode(text) + key) % n
  return unicodeToString(unicodes)

def cesar_cipher_decryption(text, key:int, n=1114112):
  unicodes = (stringToUnicode(text) - key) % n
  return unicodeToString(unicodes)

plain = 'Ensemble Learning'
encryptText = cesar_cipher_encryption('Ensemble Learning', 53)
decryptText = cesar_cipher_decryption(encryptText, 53)


print('plain text:', plain)
print('encryption:', encryptText)
print('decryption:', decryptText)

plain text: Ensemble Learning
encryption: z£¨¢¡U§££
decryption: Ensemble Learning


#### Vigenère cipher encryption & decryption
$k(x_i) = (x_i + k_{(i \mod l)}) \mod n$

$k^{-1}(x_i) = (x_i - k_{(i \mod l)}) \mod n$

In [28]:
def vigenere_cypher(type, text, key:str, n=1114112):
  key_unicodes = stringToUnicode(key)
  key_len = len(key_unicodes)
  text_unicodes = stringToUnicode(text)
  new_unicodes = []
  for i, v in enumerate(text_unicodes):
    new_v = np.nan
    if type == 'encrypt':
      new_v = (v + key_unicodes[i%key_len]) % n
    else:
      new_v = (v - key_unicodes[i%key_len]) % n
    new_unicodes.append(new_v)
  return unicodeToString(new_unicodes)

key = '5f'
plainText = 'Stochastic Variance Gradient Descent'
encryptText = vigenere_cypher('encrypt', plainText, key)
decryptText = vigenere_cypher('decrypt', encryptText, key)
encryptText, decryptText

print('plain text:', plainText)
print('encryption:', encryptText)
print('decryption:', decryptText)

plain text: Stochastic Variance Gradient Descent
encryption: Ú¤ÉÇ¨ÚÉU¼ØÇ£É|ØÊË£ÚUªÙË£Ú
decryption: Stochastic Variance Gradient Descent


#### One-Time Pad encryption & decryption

$k(x_i) = (x_i + k_i) \mod n$

$k^{-1}(x_i) = (x_i - k_i) \mod n$

In [27]:
def one_time_pad_cryption(type, text, key:str, n):
  text_unicodes = stringToUnicode(text)
  key_unicodes = stringToUnicode(key)
  new_unicodes = []
  if type == 'encrypt':
    new_unicodes = (text_unicodes + key_unicodes) % n
  else:
    new_unicodes = (text_unicodes - key_unicodes) % n
  return unicodeToString(new_unicodes)



n=256
plain = 'Hidden Markov Model'
key=generate_key(plain,n)
encryptText = one_time_pad_cryption('encrypt', plain, key, n)
decryptText = one_time_pad_cryption('decrypt', encryptText, key, n)

print('plain text:', plain)
print('encryption:', encryptText)
print('decryption:', decryptText)

plain text: Hidden Markov Model
encryption: ØnB%UB4Á8%­i`)
decryption: Hidden Markov Model


#### Lehmer pseudo random number generator

$x_{t+1}=(x_{t} \times a) \mod n$

Note: When others know current $x_{t}$, others can use formula to calculate next random number

In [29]:
class LehmerPseudoRandom:
  def __init__(self, a, n, seed):
    self.a = a
    self.n = n # compute the modolus
    self.state = seed # init x_t
  
  def get_number(self):
    self.state = (self.state * self.a) % self.n
    return self.state
  
  def get_number_below(self, exclusive_max):
    return self.get_number()%exclusive_max
  
  def get_bit(self):
    return self.get_number_below(2)

lr = LehmerPseudoRandom(a=16807, n=((2**31)-1), seed=1)
for i in range(5):
  print('random number 0-99:', lr.get_number_below(100), 'random bit:', lr.get_bit())

random number 0-99: 7 random bit: 1
random number 0-99: 73 random bit: 0
random number 0-99: 30 random bit: 0
random number 0-99: 44 random bit: 0
random number 0-99: 23 random bit: 1


#### HashMap

Hash mainly used in download error detection, hash maps/tables, and cryptography. Below is realizing hashmap.

In [3]:
class HashMap:
    
    def __init__(self, array_size):
        """Creates a hash map

        Parameters:
        array_size (int): The size of the array created to hold the data
        """       
        self.array_size = array_size 
        
        # create array containing 'None's to hold the data
        self.data = [None]*array_size
        
        # create array containing 'None's to hold the keys
        self.keys = [None]*array_size
           
    def __setitem__(self, key: str, value: any):
        """Inserts an item into the hash map.
        
        The function stores the value in the 'self.data' table
        and the key in the 'self.keys' table.
        The function will find an empty slot for the item in case of collisions.

        Parameters:
        key (string): The key to be used
        value: The value we want to store with the key
        """
        index = hash(key) % self.array_size
        loop_times = 0 # track the loop times
        while self.data[index] != None: # when meeting collisions, add 1 for null position.
          if loop_times == self.array_size: # it means the array is full
            raise IndexError('the hash map is full, please set the array_size bigger')
          index = (index+1) % self.array_size # add 1
          loop_times += 1
        self.data[index] = value
        self.keys[index] = key
        
    
    def __getitem__(self, key):
        """Retrieves an item from the hash map.
        The function will search for the item in case it has been stored 
        in a different location due to a collision.

        Parameters:
        key (string): The key to be used

        Returns:
        The item assosciated with the key.
        """
        index = hash(key) % self.array_size
        # if it has collision or the key doesn't exist
        loop_times = 0 # consider the worst situation: data is fully filled and user is checking the unexist key
        while self.keys[index] != key:
          # if key isn't exist, it will get None from keys or loop entire the keys if is fully filled.
          if self.keys[index] == None or loop_times == self.array_size: 
            return None
          index = (index + 1) % self.array_size
          loop_times += 1
        return self.data[index]

# Create a hash map of size 4
hm = HashMap(4)

# Insert numbers under their key
hm['one'] = 1
hm['two'] = 2
hm['three'] = 3
hm['four'] = 4

# Retrieve the numbers
print('four',hm['four'])
print('three',hm['three'])
print('two', hm['two'])
print('one', hm['one'])

four 4
three 3
two 2
one 1
